In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class VoxelNPZDataset(Dataset):
    def __init__(self, folder):
        self.files = sorted([
            os.path.join(folder, f)
            for f in os.listdir(folder)
            if f.endswith(".npz")
        ])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        data = np.load(self.files[idx])
        voxels = data["voxels"].astype(np.float32)  # (64,64,64)
        voxels = torch.from_numpy(voxels).unsqueeze(0)  # (1,64,64,64)
        return voxels


In [ ]:
import torch.nn as nn
class VoxelAutoEncoder(nn.Module):
    def __init__(self, latent_dim=256):
        super().__init__()

        # ---------- Encoder ----------
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=4, stride=2, padding=1),  # 64 -> 32
            nn.ReLU(),

            nn.Conv3d(32, 64, kernel_size=4, stride=2, padding=1), # 32 -> 16
            nn.ReLU(),

            nn.Conv3d(64, 128, kernel_size=4, stride=2, padding=1), # 16 -> 8
            nn.ReLU(),
        )

        self.fc_enc = nn.Linear(128 * 8 * 8 * 8, latent_dim)

        # ---------- Decoder ----------
        self.fc_dec = nn.Linear(latent_dim, 128 * 8 * 8 * 8)

        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=4, stride=2, padding=1), # 8 -> 16
            nn.ReLU(),

            nn.ConvTranspose3d(64, 32, kernel_size=4, stride=2, padding=1),  # 16 -> 32
            nn.ReLU(),

            nn.ConvTranspose3d(32, 1, kernel_size=4, stride=2, padding=1),   # 32 -> 64
            nn.Sigmoid()
        )

    def forward(self, x):
        batch_size = x.size(0)

        x = self.encoder(x)
        x = x.view(batch_size, -1)
        latent = self.fc_enc(x)

        x = self.fc_dec(latent)
        x = x.view(batch_size, 128, 8, 8, 8)
        x = self.decoder(x)

        return x, latent


In [ ]:
import torch.optim as optim
from tqdm import tqdm
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print (DEVICE)
dataset = VoxelNPZDataset("/content/drive/MyDrive/MaizeField_3D/Voxels_64")
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)

model = VoxelAutoEncoder(latent_dim=256).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()


cuda


In [ ]:
EPOCHS = 50

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0

    for voxels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        voxels = voxels.to(DEVICE)

        recon, _ = model(voxels)
        loss = criterion(recon, voxels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} | Loss: {avg_loss:.6f}")

Epoch 1/50: 100%|██████████| 262/262 [00:25<00:00, 10.16it/s]


Epoch 1 | Loss: 0.100378


Epoch 2/50: 100%|██████████| 262/262 [00:10<00:00, 24.55it/s]


Epoch 2 | Loss: 0.047209


Epoch 3/50: 100%|██████████| 262/262 [00:11<00:00, 23.24it/s]


...

Epoch 48 | Loss: 0.009299


Epoch 49/50: 100%|██████████| 262/262 [00:10<00:00, 23.82it/s]


Epoch 49 | Loss: 0.009187


Epoch 50/50: 100%|██████████| 262/262 [00:11<00:00, 23.80it/s]

Epoch 50 | Loss: 0.008995


In [ ]:
model.eval()

# Load one voxel file
input_npz = "/content/drive/MyDrive/MaizeField_3D/Voxels_64/0004_vox64.npz"
data = np.load(input_npz)
voxels = torch.from_numpy(data["voxels"]).float().unsqueeze(0).unsqueeze(0).to(DEVICE)

with torch.no_grad():
    recon, latent = model(voxels)

# Threshold output
recon_voxels = (recon.squeeze().cpu().numpy() > 0.5).astype(np.uint8)

# Save reconstructed voxel grid
recon_npz_path = "/content/sample_data/reconstructed_voxel.npz"
np.savez_compressed(recon_npz_path, voxels=recon_voxels)

print("Reconstructed voxel saved to:", recon_npz_path)


Reconstructed voxel saved to: /content/sample_data/reconstructed_voxel.npz


In [ ]:
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 150.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
import open3d as o3d
# Load reconstructed voxel grid
data = np.load(recon_npz_path)
voxels = data["voxels"]  # (64,64,64)

# Get occupied voxel coordinates
points = np.argwhere(voxels > 0)

# Normalize for visualization
points = points.astype(np.float32)
points /= np.max(points)

# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.paint_uniform_color([0.1, 0.8, 0.1])

# Save as PLY
ply_out = "/content/sample_data/reconstructed_voxel.ply"
o3d.io.write_point_cloud(ply_out, pcd)

print("Reconstructed PLY saved to:", ply_out)


Reconstructed PLY saved to: /content/sample_data/reconstructed_voxel.ply
